In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.optim as optim
torch.manual_seed(1)

import random
random.seed(1)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w7-p1/mnist_test.csv
/kaggle/input/2021-ai-w7-p1/mnist_train.csv
/kaggle/input/2021-ai-w7-p1/submission.csv


In [2]:
X = pd.read_csv('/kaggle/input/2021-ai-w7-p1/mnist_train.csv')
y = X['784']
X = X.drop(['Unnamed: 0', '784'], axis = 1)
test = pd.read_csv('/kaggle/input/2021-ai-w7-p1/mnist_test.csv').drop(['Unnamed: 0'], axis = 1)
sample = pd.read_csv('/kaggle/input/2021-ai-w7-p1/submission.csv')

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

if device == 'cuda':
    torch.cuda.manual_seed_all(1)

In [4]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
X_sc = sc.fit_transform(X)
test_sc = sc.transform(test)

In [5]:
# X = torch.FloatTensor(X.to_numpy()).to(device)
# test = torch.FloatTensor(test.to_numpy()).to(device)
y = torch.LongTensor(y.to_numpy()).to(device)

In [6]:
X_sc = torch.FloatTensor(X_sc).to(device)
test_sc = torch.FloatTensor(test_sc).to(device)

In [7]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size=0.33, random_state=42)

In [8]:
linear1 = torch.nn.Linear(784, 256)
linear2 = torch.nn.Linear(256, 256)
linear3 = torch.nn.Linear(256, 256)
linear4 = torch.nn.Linear(256, 10)
linear5 = torch.nn.Linear(512, 10)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=0.4)

torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)
torch.nn.init.xavier_normal_(linear3.weight)
torch.nn.init.xavier_normal_(linear4.weight)
# torch.nn.init.xavier_uniform_(linear5.weight)

model = torch.nn.Sequential(linear1, relu, dropout,
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4).to(device)

In [9]:
# from torch.utils.data import TensorDataset
# dataset = TensorDataset(X_sc, y)
# dataloader = torch.utils.data.DataLoader(dataset,shuffle=True,batch_size=500, drop_last = True)

In [10]:
loss = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

for epoch in range(501):
    hypothesis = model(X_sc)
    cost = loss(hypothesis, y)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch%50==0:
        print(epoch, cost.item())

0 2.4053337574005127
50 0.27694734930992126
100 0.14853698015213013
150 0.09709536284208298
200 0.07076843827962875
250 0.049580469727516174
300 0.03899148106575012
350 0.030989961698651314
400 0.024271314963698387
450 0.019564973190426826
500 0.01728806085884571


In [11]:
# model.train()
# loss = torch.nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr = 1e-4)

# for epoch in range(121):
#     for x_train, y_train in dataloader:
#         hypothesis = model(x_train)
#         cost = loss(hypothesis, y_train)
#         optimizer.zero_grad()
#         cost.backward()
#         optimizer.step()
#     if epoch%10==0:
#             print(epoch, cost.item())

In [12]:
model.eval()
with torch.no_grad():
    predict = torch.nn.functional.softmax(model(X_sc), dim=1)
    predict = torch.argmax(predict, dim=1)
    correct = y == predict.float()
    accuracy = correct.sum().item()/len(correct)
    print(accuracy)
#0.9999666666666667

0.9999166666666667


In [13]:
model.eval()
with torch.no_grad():
    predict = torch.nn.functional.softmax(model(test_sc), dim=1)
    predict = torch.argmax(predict, dim=1)
    print(predict.type)

<built-in method type of Tensor object at 0x7fb70564e5f0>


In [14]:
predict = predict.cpu().numpy()
predict

array([7, 2, 1, ..., 4, 5, 6])

In [15]:
sample['Label'] = predict
sample.head()

,id,Label
0,0,7
1,1,2
2,2,1
3,3,0
4,4,4


In [16]:
sample.to_csv("submit_none.csv", index=False)